=== Lấy cấu trúc khóa học (courseid=3) ===

=== Danh sách các section trong khóa học (courseid=3) ===


Section ID,Tên chương,Tóm tắt,Thứ tự,Hiển thị,Component,Item ID
9,General,,0,Có,None,NaN
10,Chương 1: Lập trình Java căn bản,,1,Có,None,NaN
11,Chương 2: Lập trình Hướng đối tượng với Java,,2,Có,None,NaN
12,Chương 3: Lập trình Giao diện đồ họa (Winform),,3,Có,None,NaN
13,Chương 4: Kết nối Cơ sở dữ liệu,,4,Có,None,NaN
14,Chương 5: Giới thiệu JSP và ServLet,,5,Có,None,NaN
17,Lớp và đối tượng,,6,Có,mod_subsection,5.0
18,Tính kế thừa và đa hình,,7,Có,mod_subsection,6.0
19,Cấu trúc chương trình và biến,\nTìm hiểu cấu trúc cơ bản của chương trình Java và cách sử dụng biến.\n,8,Có,mod_subsection,7.0
20,Câu lệnh điều kiện và vòng lặp,\nHọc cách sử dụng các cấu trúc điều khiển luồng như câu lệnh điều kiện và vòng lặp\n,9,Có,mod_subsection,8.0


In [5]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "ed3520980c2a735542b59da43e98188f"

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API ({wsfunction} với params={params}): {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu ({wsfunction}): {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON ({wsfunction}): {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def clean_html(html_text):
    """
    Làm sạch HTML để lấy văn bản thuần túy.

    Args:
        html_text (str): Chuỗi HTML.

    Returns:
        str: Văn bản đã làm sạch.
    """
    return BeautifulSoup(html_text, 'html.parser').get_text() if html_text else ""

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    print(f"Đang lấy nội dung khóa học courseid={courseid}...")
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

def get_section_structure(courseid):
    """
    Lấy cấu trúc section và xác định subsection thuộc section chính nào.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        dict: Cấu trúc lồng nhau với section chính và subsection.
    """
    sections = get_course_contents(courseid)
    if not sections:
        print(f"❌ Không lấy được section cho khóa học {courseid}.")
        return None
    
    structure = {
        'courseid': courseid,
        'sections': []
    }
    
    # Tạo ánh xạ subsection ID tới section chính
    subsection_to_parent = {}
    for section in sections:
        for module in section.get('modules', []):
            if module['modname'] == 'subsection':
                subsection_to_parent[module['instance']] = section['id']
    
    for section in sections:
        section_info = {
            'id': section['id'],
            'name': section.get('name', 'Không có tên'),
            'summary': clean_html(section.get('summary', '')),
            'section_order': section.get('section', 0),
            'visible': 'Có' if section.get('visible', 0) == 1 else 'Không',
            'component': section.get('component', 'N/A'),
            'item_id': section.get('itemid', 'N/A'),
            'subsections': [],
            'is_subsection': section['id'] in subsection_to_parent,
            'parent_section_id': subsection_to_parent.get(section['id'], None)
        }
        
        # Tìm subsections thuộc section này
        for module in section.get('modules', []):
            if module['modname'] == 'subsection':
                print(f"Đang lấy nội dung subsection instance={module['instance']}...")
                subsection_data = get_course_contents(module['instance'])
                if subsection_data:
                    for sub_section in subsection_data:
                        subsection_info = {
                            'id': sub_section['id'],
                            'name': sub_section.get('name', 'Không có tên'),
                            'summary': clean_html(sub_section.get('summary', '')),
                            'section_order': sub_section.get('section', 0),
                            'visible': 'Có' if sub_section.get('visible', 0) == 1 else 'Không',
                            'component': sub_section.get('component', 'N/A'),
                            'item_id': sub_section.get('itemid', 'N/A')
                        }
                        section_info['subsections'].append(subsection_info)
                else:
                    print(f"⚠️ Không lấy được nội dung cho subsection instance={module['instance']}.")
        
        structure['sections'].append(section_info)
    
    return structure

def display_section_structure(courseid):
    """
    Hiển thị cấu trúc section và subsection dưới dạng bảng.

    Args:
        courseid (int): ID của khóa học.
    """
    structure = get_section_structure(courseid)
    if not structure:
        return
    
    print(f"\n=== Cấu trúc section trong khóa học (courseid={courseid}) ===")
    
    # Chuẩn bị dữ liệu cho bảng
    section_data = []
    for section in structure['sections']:
        section_data.append({
            'Section ID': section['id'],
            'Tên chương': section['name'],
            'Tóm tắt': section['summary'],
            'Thứ tự': section['section_order'],
            'Hiển thị': section['visible'],
            'Component': section['component'],
            'Item ID': section['item_id'],
            'Section chính': 'N/A' if not section['is_subsection'] else section['parent_section_id']
        })
        
        for subsection in section['subsections']:
            section_data.append({
                'Section ID': subsection['id'],
                'Tên chương': f"└── {subsection['name']}",
                'Tóm tắt': subsection['summary'],
                'Thứ tự': subsection['section_order'],
                'Hiển thị': subsection['visible'],
                'Component': subsection['component'],
                'Item ID': subsection['item_id'],
                'Section chính': section['id']
            })
    
    # Hiển thị bảng
    df = pd.DataFrame(section_data)
    display(HTML(df.to_html(index=False)))

# Kiểm tra danh sách khóa học
def check_available_courses():
    """
    Lấy và hiển thị danh sách tất cả khóa học.
    """
    courses = call_moodle_api('core_course_get_courses')
    if courses:
        print("\n=== Danh sách khóa học ===")
        course_data = [{'ID': c['id'], 'Tên khóa học': c['fullname']} for c in courses]
        df = pd.DataFrame(course_data)
        display(HTML(df.to_html(index=False)))
    else:
        print("❌ Không lấy được danh sách khóa học.")

# Gọi hàm để kiểm tra và hiển thị
if __name__ == "__main__":
    print("=== Kiểm tra danh sách khóa học ===")
    check_available_courses()
    
    courseid = 3  # Khóa học "NGÔN NGỮ LẬP TRÌNH JAVA"
    print(f"\n=== Lấy cấu trúc khóa học (courseid={courseid}) ===")
    display_section_structure(courseid)

=== Kiểm tra danh sách khóa học ===

=== Danh sách khóa học ===


ID,Tên khóa học
1,Nguyen Huu Loc
2,Tin học 12
3,NGÔN NGỮ LẬP TRÌNH JAVA



=== Lấy cấu trúc khóa học (courseid=3) ===
Đang lấy nội dung khóa học courseid=3...
Đang lấy nội dung subsection instance=7...
Đang lấy nội dung khóa học courseid=7...
❌ Lỗi API (core_course_get_contents với params={'courseid': 7}): invalidrecordunknown
Thông điệp: Can't find data record in database.
⚠️ Không lấy được nội dung cho subsection instance=7.
Đang lấy nội dung subsection instance=8...
Đang lấy nội dung khóa học courseid=8...
❌ Lỗi API (core_course_get_contents với params={'courseid': 8}): invalidrecordunknown
Thông điệp: Can't find data record in database.
⚠️ Không lấy được nội dung cho subsection instance=8.
Đang lấy nội dung subsection instance=5...
Đang lấy nội dung khóa học courseid=5...
❌ Lỗi API (core_course_get_contents với params={'courseid': 5}): invalidrecordunknown
Thông điệp: Can't find data record in database.
⚠️ Không lấy được nội dung cho subsection instance=5.
Đang lấy nội dung subsection instance=6...
Đang lấy nội dung khóa học courseid=6...
❌ Lỗi API (core

Section ID,Tên chương,Tóm tắt,Thứ tự,Hiển thị,Component,Item ID,Section chính
9,General,,0,Có,None,NaN,N/A
10,Chương 1: Lập trình Java căn bản,,1,Có,None,NaN,N/A
11,Chương 2: Lập trình Hướng đối tượng với Java,,2,Có,None,NaN,N/A
12,Chương 3: Lập trình Giao diện đồ họa (Winform),,3,Có,None,NaN,N/A
13,Chương 4: Kết nối Cơ sở dữ liệu,,4,Có,None,NaN,N/A
14,Chương 5: Giới thiệu JSP và ServLet,,5,Có,None,NaN,N/A
17,Lớp và đối tượng,,6,Có,mod_subsection,5.0,N/A
18,Tính kế thừa và đa hình,,7,Có,mod_subsection,6.0,N/A
19,Cấu trúc chương trình và biến,\nTìm hiểu cấu trúc cơ bản của chương trình Java và cách sử dụng biến.\n,8,Có,mod_subsection,7.0,N/A
20,Câu lệnh điều kiện và vòng lặp,\nHọc cách sử dụng các cấu trúc điều khiển luồng như câu lệnh điều kiện và vòng lặp\n,9,Có,mod_subsection,8.0,N/A


In [6]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "ed3520980c2a735542b59da43e98188f"

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API ({wsfunction} với params={params}): {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu ({wsfunction}): {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON ({wsfunction}): {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def clean_html(html_text):
    """
    Làm sạch HTML để lấy văn bản thuần túy.

    Args:
        html_text (str): Chuỗi HTML.

    Returns:
        str: Văn bản đã làm sạch.
    """
    return BeautifulSoup(html_text, 'html.parser').get_text() if html_text else ""

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    print(f"Đang lấy nội dung khóa học courseid={courseid}...")
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

def get_section_structure(courseid):
    """
    Lấy cấu trúc section và xác định subsection thuộc section chính nào.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        dict: Cấu trúc lồng nhau với section chính và subsection.
    """
    sections = get_course_contents(courseid)
    if not sections:
        print(f"❌ Không lấy được section cho khóa học {courseid}.")
        return None
    
    structure = {
        'courseid': courseid,
        'sections': []
    }
    
    # Tạo ánh xạ subsection ID tới section chính
    subsection_to_parent = {}
    for section in sections:
        for module in section.get('modules', []):
            if module['modname'] == 'subsection':
                subsection_to_parent[module['instance']] = section['id']
    
    # Xây dựng cấu trúc section
    for section in sections:
        section_info = {
            'id': section['id'],
            'name': section.get('name', 'Không có tên'),
            'summary': clean_html(section.get('summary', '')),
            'section_order': section.get('section', 0),
            'visible': 'Có' if section.get('visible', 0) == 1 else 'Không',
            'component': section.get('component', 'N/A'),
            'item_id': section.get('itemid', 'N/A'),
            'subsections': [],
            'is_subsection': section['id'] in subsection_to_parent,
            'parent_section_id': subsection_to_parent.get(section['id'], None)
        }
        
        # Tìm subsections thuộc section này
        for module in section.get('modules', []):
            if module['modname'] == 'subsection':
                # Tìm section tương ứng với instance của module
                subsection_id = module['instance']
                subsection = next((s for s in sections if s['id'] == subsection_id), None)
                if subsection:
                    subsection_info = {
                        'id': subsection['id'],
                        'name': subsection.get('name', 'Không có tên'),
                        'summary': clean_html(subsection.get('summary', '')),
                        'section_order': subsection.get('section', 0),
                        'visible': 'Có' if subsection.get('visible', 0) == 1 else 'Không',
                        'component': subsection.get('component', 'N/A'),
                        'item_id': subsection.get('itemid', 'N/A')
                    }
                    section_info['subsections'].append(subsection_info)
                else:
                    print(f"⚠️ Không tìm thấy subsection với ID={subsection_id} trong dữ liệu khóa học.")
        
        structure['sections'].append(section_info)
    
    return structure

def display_section_structure(courseid):
    """
    Hiển thị cấu trúc section và subsection dưới dạng bảng.

    Args:
        courseid (int): ID của khóa học.
    """
    structure = get_section_structure(courseid)
    if not structure:
        return
    
    print(f"\n=== Cấu trúc section trong khóa học (courseid={courseid}) ===")
    
    # Chuẩn bị dữ liệu cho bảng
    section_data = []
    for section in structure['sections']:
        section_data.append({
            'Section ID': section['id'],
            'Tên chương': section['name'],
            'Tóm tắt': section['summary'],
            'Thứ tự': section['section_order'],
            'Hiển thị': section['visible'],
            'Component': section['component'],
            'Item ID': section['item_id'],
            'Section chính': 'N/A' if not section['is_subsection'] else section['parent_section_id']
        })
        
        for subsection in section['subsections']:
            section_data.append({
                'Section ID': subsection['id'],
                'Tên chương': f"└── {subsection['name']}",
                'Tóm tắt': subsection['summary'],
                'Thứ tự': subsection['section_order'],
                'Hiển thị': subsection['visible'],
                'Component': subsection['component'],
                'Item ID': subsection['item_id'],
                'Section chính': section['id']
            })
    
    # Hiển thị bảng
    df = pd.DataFrame(section_data)
    display(HTML(df.to_html(index=False)))

# Kiểm tra danh sách khóa học
def check_available_courses():
    """
    Lấy và hiển thị danh sách tất cả khóa học.
    """
    courses = call_moodle_api('core_course_get_courses')
    if courses:
        print("\n=== Danh sách khóa học ===")
        course_data = [{'ID': c['id'], 'Tên khóa học': c['fullname']} for c in courses]
        df = pd.DataFrame(course_data)
        display(HTML(df.to_html(index=False)))
    else:
        print("❌ Không lấy được danh sách khóa học.")

# Gọi hàm để kiểm tra và hiển thị
if __name__ == "__main__":
    print("=== Kiểm tra danh sách khóa học ===")
    check_available_courses()
    
    courseid = 3  # Khóa học "NGÔN NGỮ LẬP TRÌNH JAVA"
    print(f"\n=== Lấy cấu trúc khóa học (courseid={courseid}) ===")
    display_section_structure(courseid)

=== Kiểm tra danh sách khóa học ===

=== Danh sách khóa học ===


ID,Tên khóa học
1,Nguyen Huu Loc
2,Tin học 12
3,NGÔN NGỮ LẬP TRÌNH JAVA



=== Lấy cấu trúc khóa học (courseid=3) ===
Đang lấy nội dung khóa học courseid=3...
⚠️ Không tìm thấy subsection với ID=7 trong dữ liệu khóa học.
⚠️ Không tìm thấy subsection với ID=8 trong dữ liệu khóa học.
⚠️ Không tìm thấy subsection với ID=5 trong dữ liệu khóa học.
⚠️ Không tìm thấy subsection với ID=6 trong dữ liệu khóa học.

=== Cấu trúc section trong khóa học (courseid=3) ===


Section ID,Tên chương,Tóm tắt,Thứ tự,Hiển thị,Component,Item ID,Section chính
9,General,,0,Có,None,NaN,N/A
10,Chương 1: Lập trình Java căn bản,,1,Có,None,NaN,N/A
11,Chương 2: Lập trình Hướng đối tượng với Java,,2,Có,None,NaN,N/A
12,Chương 3: Lập trình Giao diện đồ họa (Winform),,3,Có,None,NaN,N/A
13,Chương 4: Kết nối Cơ sở dữ liệu,,4,Có,None,NaN,N/A
14,Chương 5: Giới thiệu JSP và ServLet,,5,Có,None,NaN,N/A
17,Lớp và đối tượng,,6,Có,mod_subsection,5.0,N/A
18,Tính kế thừa và đa hình,,7,Có,mod_subsection,6.0,N/A
19,Cấu trúc chương trình và biến,\nTìm hiểu cấu trúc cơ bản của chương trình Java và cách sử dụng biến.\n,8,Có,mod_subsection,7.0,N/A
20,Câu lệnh điều kiện và vòng lặp,\nHọc cách sử dụng các cấu trúc điều khiển luồng như câu lệnh điều kiện và vòng lặp\n,9,Có,mod_subsection,8.0,N/A
